# Radiative transfer model of HD209458b

Main reference: https://www.aanda.org/articles/aa/full_html/2010/12/aa13396-09/aa13396-09.html

## Question 1

We reformulate the FTE using pressure as the coordinate. The FTE

$$ \mu \frac{dI}{dz} = \rho \kappa (B(T(z)) - I) $$

Using the chain rule, we have

$$ \mu \frac{dI}{dP} \frac{dP}{dz} = \rho \kappa (B(T(z)) - I) $$

Substituting the equation for hydrostatic equilibrium $\frac{dP}{dz} = -g \rho$, we get

$$ -\mu \frac{dI}{dP} = \frac{\kappa}{g} (B(T(P)) - I) $$

Working in the P-coordinate has a few advantages:
1. No need to know the equation of state of the gas
2. Gas elements stays glued to their value of P, even if the temperature $T(P)$ changes

In [2]:
# some relevant physical quantities

def rho(z):
    """
    Gives the gas density of the atmosphere assuming semi-infinite space
    Need: rho0
    """
    res = rho0 if z <= 0 else 0
    
    return res

def alpha(z):
    """
    extinction function
    Need: kappa (opacity)
    """
    return rho(z) * kappa

def source(z):
    """
    Source function
    Need: eta, planck function B, mean intensity J
    """
    return (1-eta) * B + eta * J

def fte(z, I, mu):
    """
    Gives the derivative of I wrt position z (RHS of the ODE).
    """
    return alpha(z)/mu * (source(z) - I)



**Step 1**:
$$ F_*(P = P_{min}) = \frac{L_*}{4\pi d^2} $$
Now, integrate $F_*$ downward under angle $i$. The ODE is:
$$ \frac{dF_*(P)}{dP} = -\frac{K_*}{g \cos(i)} F_*(P)$$

where $k_* = 4 \times 10^{-3} cm^2/g = 4 \times 10^{-4} m^2/kg$ and $g = 9.66 m/s^2$. Loop:
$$ \Delta\tau = \frac{K_*}{g\cos(i)} F_*(P)$$
$$ F_*[i-1] = F_*[i] * \exp(-\Delta\tau) $$

And then $J_* = \frac{F_*}{4\pi}$, i.e. the mean intensity of the stellar light.

**Step 2**: Lambda iteration
* Start with initial guess of $T(P)$, e.g. $T(P)$ = 2000 K.
* Loop iteration:
    - Compute $B(T) = \frac{\sigma_*}{\pi} * T^4$
    - `B[:] = (sigma_b/pi) * T[:]**4`
    - Source function: (take albedo = 0)
    - `S[:] = B[:]`

Moments of intensity
$$ J = \frac{1}{4\pi} \oint I(\Omega) \,d\Omega = \frac{1}{2} \int_{-1}^{+1} I_{\mu}\,d\mu$$

mean intensity = 0th moment (scalar), 1st moment (vector), 2nd moment (rank 2 tensor).
$$ \frac{1}{4\pi} \oint \vec{n} \cdot \vec{\nabla} I \,d\Omega = \frac{1}{4\pi} \oint \alpha (S - I) \,d\Omega$$
$$ \frac{1}{4\pi}\vec{\nabla} \cdot \oint \vec{n}  I \,d\Omega = \alpha S - \frac{1}{4\pi} \alpha \oint{I \,d\Omega} = \alpha (S - T)$$
$$ \frac{1}{4\pi} \nabla \cdot \oint \vec{n} \vec{n} I \,d\Omega = - \frac{\alpha}{4\pi} \oint I \vec{n} \,d\Omega$$

In [4]:
from scipy.integrate import solve_ivp
import numpy as np

In [3]:
# setup the test problem
B = 1
rho0 = 1
kappa = 1
eta = 0

z_grid = np.linspace(-10, 0, 100)
mu_grid = np.linspace(-1, 1, 20) # no mu at zero
I_grid = np.zeros(z_grid.size)
